In [10]:
import uproot
import awkward as ak
import numpy as np
import math
import hist
import matplotlib.pyplot as plt
import os
import subprocess
import vector
import gc
import pandas as pd

In [112]:
l1 = []
l2 = []

l1 = pd.read_csv("../data/counters.offline.csv")["run"]
l2 = [int(e) for e in os.popen("ls /pbs/throng/training/nantes-m2-rps-exp/data/ | grep 'mc' | cut -c 4-9").read().split("\n")[0:-2]]
#list(set(l1) & set(l2))

BASEDIR = "/pbs/throng/training/nantes-m2-rps-exp/data"
DATATYPE = ["mc", "mc", "data"]
TREE = ["gen", "events", "events"]

l3 = []
for l in l2:
    for batch in uproot.iterate(f"/pbs/throng/training/nantes-m2-rps-exp/data/run{l}.mc.root:genTree",                                
                                report = True):

        events = batch[0]
        if len(events) < 1000:
            #print("something is wrong", batch[1])
            break
            
        l3 = np.append(l3, l)
        
_ = list(set(l1) & set(l2) & set(l3))

SAMPLE_RUNS = [int(run) for run in _]

#print(SAMPLE_RUNS)

In [113]:
on =  pd.read_csv('../data/counters.online.csv')
off = pd.read_csv('../data/counters.offline.csv')

In [114]:
#----------METHODE ONLINE----------

cint7l0b = np.array([], dtype=float)
cmul7l0b = np.array([], dtype=float)
duration = np.array([], dtype=float) 
F_NORM_online = np.array([], dtype=float)
N_MB_online = np.array([], dtype=float)


for i in SAMPLE_RUNS: 
    ligne_cible_on = on.loc[on["run"] == i]
    
    cint7l0b = np.append(cint7l0b, ligne_cible_on["cint7l0b"].values[0])
    cmul7l0b = np.append(cmul7l0b, ligne_cible_on["cmul7l0b"].values[0])
    duration = np.append(duration, ligne_cible_on["duration(s)"].values[0])
    F_NORM_online = np.append(F_NORM_online, ligne_cible_on["cint7l0b"].values[0]/ligne_cible_on["cmul7l0b"].values[0])

for i in range(len(F_NORM_online)):
    N_MB_online = np.append(N_MB_online, F_NORM_online[i]*cmul7l0b[i])

In [115]:
#----------METHODE OFFLINE----------

#----Données ALL---- : 

cint7all = np.array([], dtype=float)
cmul7all = np.array([], dtype=float)
cmsl7all = np.array([], dtype=float)
F_NORM_offline_all = np.array([], dtype=float)
N_MB_offline_all = np.array([], dtype=float)

for i in SAMPLE_RUNS: 
    ligne_cible_off = off.loc[off["run"] == i]
    cint7all = np.append(cint7all, ligne_cible_off["cint7all"].values[0])
    cmul7all = np.append(cmul7all, ligne_cible_off["cmul7all"].values[0])
    cmsl7all = np.append(cmsl7all, ligne_cible_off["cmsl7all"].values[0])
    F_NORM_offline_all = np.append(F_NORM_offline_all, (ligne_cible_off["cmsl7all"].values[0]/ligne_cible_off["cmsl7all&0mul"].values[0])*
                                  (ligne_cible_off["cint7all"].values[0]/ligne_cible_off["cint7all&0msl"].values[0]))
                                  
for i in range(len(F_NORM_offline_all)):
    N_MB_offline_all = np.append(N_MB_offline_all, F_NORM_offline_all[i]*cmul7all[i])

#print(cmul7all)
#print(F_NORM_offline_all)





In [116]:
#----------METHODE OFFLINE----------

#----Données PS---- : 

cint7ps = np.array([], dtype=float)
cmul7ps = np.array([], dtype=float)
cmsl7ps = np.array([], dtype=float)
F_NORM_offline_ps = np.array([], dtype=float)
N_MB_offline_ps = np.array([], dtype=float)

for i in SAMPLE_RUNS: 
    ligne_cible_off = off.loc[off["run"] == i]
    cint7ps = np.append(cint7ps, ligne_cible_off["cint7ps"].values[0])
    cmul7ps = np.append(cmul7ps, ligne_cible_off["cmul7ps"].values[0])
    cmsl7ps = np.append(cmsl7ps, ligne_cible_off["cmsl7ps"].values[0])
    F_NORM_offline_ps = np.append(F_NORM_offline_ps, (ligne_cible_off["cmsl7ps"].values[0]/ligne_cible_off["cmsl7all&0mul"].values[0])*
                                  (ligne_cible_off["cint7ps"].values[0]/ligne_cible_off["cint7all&0msl"].values[0]))
                                  
for i in range(len(F_NORM_offline_ps)):
    N_MB_offline_ps = np.append(N_MB_offline_ps, F_NORM_offline_ps[i]*cmul7ps[i])

#print(F_NORM_offline_ps)



#Calcule du N_MB moyen pondéré par différentes données :

#0- Pas de pondération

moy_N_MB_simple_off_ps = np.sum(N_MB_offline_ps)/len(N_MB_offline_ps)


#1- CINT PS

N_MBxcint7ps_off = np.array([], dtype=float)
for i in range(len(N_MB_offline_ps)):
    N_MBxcint7ps_off = np.append(N_MBxcint7ps_off, N_MB_offline_ps[i]*cint7ps[i])

moy_N_MB_cint7ps = np.sum(N_MBxcint7ps_off)/np.sum(cint7ps)


#2- CMUL PS

N_MBxcmul7ps_off = np.array([], dtype=float)
for i in range(len(N_MB_offline_ps)):
    N_MBxcmul7ps_off = np.append(N_MBxcmul7ps_off, N_MB_offline_ps[i]*cmul7ps[i])

moy_N_MB_cmul7ps = np.sum(N_MBxcmul7ps_off)/np.sum(cmul7ps)


#3- CMSL PS

N_MBxcmsl7ps_off = np.array([], dtype=float)
for i in range(len(N_MB_offline_ps)):
    N_MBxcmsl7ps_off = np.append(N_MBxcmsl7ps_off, N_MB_offline_ps[i]*cmsl7ps[i])

moy_N_MB_cmsl7ps = np.sum(N_MBxcmsl7ps_off)/np.sum(cmsl7ps)


#4- Duration

N_MBxduration_off_ps = np.array([], dtype=float)
for i in range(len(N_MB_offline_ps)):
    N_MBxduration_off_ps = np.append(N_MBxduration_off_ps, N_MB_offline_ps[i]*duration[i])

moy_N_MB_duration_off_ps = np.sum(N_MBxduration_off_ps)/np.sum(duration)



print("Valeur des différentes moyennes de N_MB_off_ps (méthode offline seulement) : \n",
      "\n moyenne non pondéré de N_MB_off_ps             :", moy_N_MB_simple_off_ps,
      "\n moyenne pondéré de N_MB_off_ps par cint7ps     :", moy_N_MB_cint7ps,
      "\n moyenne pondéré de N_MB_off_ps par cmul7ps     :", moy_N_MB_cmul7ps,
      "\n moyenne pondéré de N_MB_off_ps par cmsl7ps     :", moy_N_MB_cmsl7ps,
      "\n moyenne pondéré de N_MB_off_ps par duration(s) :", moy_N_MB_duration_off_ps,)

Valeur des différentes moyennes de N_MB_off_ps (méthode offline seulement) : 
 
 moyenne non pondéré de N_MB_off_ps             : 822857743.946547 
 moyenne pondéré de N_MB_off_ps par cint7ps     : 1190108574.4197583 
 moyenne pondéré de N_MB_off_ps par cmul7ps     : 1192477695.4922519 
 moyenne pondéré de N_MB_off_ps par cmsl7ps     : 1193010778.554724 
 moyenne pondéré de N_MB_off_ps par duration(s) : 1187458378.5668173


In [117]:
########################### CALCUL MOYENNE ###########################

#--------- METHODE ONLINE ---------#

#0- Pas de pondération

moy_N_MB_simple_on = np.sum(N_MB_online)/len(N_MB_online)


#1- CINT

N_MBxcint7l0b_on = np.array([], dtype=float)
for i in range(len(N_MB_online)):
    N_MBxcint7l0b_on = np.append(N_MBxcint7l0b_on, N_MB_online[i]*cint7l0b[i])

moy_N_MB_cint7l0b_on = np.sum(N_MBxcint7l0b_on)/np.sum(cint7l0b)


#2- CMUL 

N_MBxcmul7l0b_on = np.array([], dtype=float)
for i in range(len(N_MB_online)):
    N_MBxcmul7l0b_on = np.append(N_MBxcmul7l0b_on, N_MB_online[i]*cmul7l0b[i])

moy_N_MB_cmul7l0b_on = np.sum(N_MBxcmul7l0b_on)/np.sum(cmul7l0b)


#3- Duration

N_MBxduration_on = np.array([], dtype=float)
for i in range(len(N_MB_online)):
    N_MBxduration_on = np.append(N_MBxduration_on, N_MB_online[i]*duration[i])

moy_N_MB_duration_on = np.sum(N_MBxduration_on)/np.sum(duration)




#--------- METHODE OFFLINE ALL ---------#

#0- Pas de pondération

moy_N_MB_simple_off_all = np.sum(N_MB_offline_all)/len(N_MB_offline_all)


#1- CINT ALL

N_MBxcint7all_off = np.array([], dtype=float)
for i in range(len(N_MB_offline_all)):
    N_MBxcint7all_off = np.append(N_MBxcint7all_off, N_MB_offline_all[i]*cint7all[i])

moy_N_MB_cint7all = np.sum(N_MBxcint7all_off)/np.sum(cint7all)


#2- CMUL ALL

N_MBxcmul7all_off = np.array([], dtype=float)
for i in range(len(N_MB_offline_all)):
    N_MBxcmul7all_off = np.append(N_MBxcmul7all_off, N_MB_offline_all[i]*cmul7all[i])

moy_N_MB_cmul7all = np.sum(N_MBxcmul7all_off)/np.sum(cmul7all)


#3- CMSL ALL

N_MBxcmsl7all_off = np.array([], dtype=float)
for i in range(len(N_MB_offline_all)):
    N_MBxcmsl7all_off = np.append(N_MBxcmsl7all_off, N_MB_offline_all[i]*cmsl7all[i])

moy_N_MB_cmsl7all = np.sum(N_MBxcmsl7all_off)/np.sum(cmsl7all)


#4- Duration

N_MBxduration_off_all = np.array([], dtype=float)
for i in range(len(N_MB_offline_all)):
    N_MBxduration_off_all = np.append(N_MBxduration_off_all, N_MB_offline_all[i]*duration[i])

moy_N_MB_duration_off_all = np.sum(N_MBxduration_off_all)/np.sum(duration)




#--------- METHODE OFFLINE PS ---------#

#0- Pas de pondération

moy_N_MB_simple_off_ps = np.sum(N_MB_offline_ps)/len(N_MB_offline_ps)


#1- CINT PS

N_MBxcint7ps_off = np.array([], dtype=float)
for i in range(len(N_MB_offline_ps)):
    N_MBxcint7ps_off = np.append(N_MBxcint7ps_off, N_MB_offline_ps[i]*cint7ps[i])

moy_N_MB_cint7ps = np.sum(N_MBxcint7ps_off)/np.sum(cint7ps)


#2- CMUL PS

N_MBxcmul7ps_off = np.array([], dtype=float)
for i in range(len(N_MB_offline_ps)):
    N_MBxcmul7ps_off = np.append(N_MBxcmul7ps_off, N_MB_offline_ps[i]*cmul7ps[i])

moy_N_MB_cmul7ps = np.sum(N_MBxcmul7ps_off)/np.sum(cmul7ps)


#3- CMSL PS

N_MBxcmsl7ps_off = np.array([], dtype=float)
for i in range(len(N_MB_offline_ps)):
    N_MBxcmsl7ps_off = np.append(N_MBxcmsl7ps_off, N_MB_offline_ps[i]*cmsl7ps[i])

moy_N_MB_cmsl7ps = np.sum(N_MBxcmsl7ps_off)/np.sum(cmsl7ps)


#4- Duration

N_MBxduration_off_ps = np.array([], dtype=float)
for i in range(len(N_MB_offline_ps)):
    N_MBxduration_off_ps = np.append(N_MBxduration_off_ps, N_MB_offline_ps[i]*duration[i])

moy_N_MB_duration_off_ps = np.sum(N_MBxduration_off_ps)/np.sum(duration)

In [121]:
print("Valeur des différentes moyennes de N_MB_on (méthode online seulement) : \n",
      "\n moyenne non pondéré de N_MB_on             :", moy_N_MB_simple_on,
      "\n moyenne pondéré de N_MB_on par cint7l0b    :", moy_N_MB_cint7l0b_on,
      "\n moyenne pondéré de N_MB_on par cmul7l0b    :", moy_N_MB_cmul7l0b_on,
      "\n moyenne pondéré de N_MB_on par duration(s) :", moy_N_MB_duration_on,"\n \n")

print("Valeur des différentes moyennes de N_MB_off_all (méthode offline all + duration) : \n",
      "\n moyenne non pondéré de N_MB_off_all             :", moy_N_MB_simple_off_all,
      "\n moyenne pondéré de N_MB_off_all par cint7all    :", moy_N_MB_cint7all,
      "\n moyenne pondéré de N_MB_off_all par cmul7all    :", moy_N_MB_cmul7all,
      "\n moyenne pondéré de N_MB_off_all par cmsl7all    :", moy_N_MB_cmsl7all,
      "\n moyenne pondéré de N_MB_off_all par duration(s) :", moy_N_MB_duration_off_all,"\n \n")

print("Valeur des différentes moyennes de N_MB_off_ps (méthode offline ps + duration) : \n",
      "\n moyenne non pondéré de N_MB_off_ps             :", moy_N_MB_simple_off_ps,
      "\n moyenne pondéré de N_MB_off_ps par cint7ps     :", moy_N_MB_cint7ps,
      "\n moyenne pondéré de N_MB_off_ps par cmul7ps     :", moy_N_MB_cmul7ps,
      "\n moyenne pondéré de N_MB_off_ps par cmsl7ps     :", moy_N_MB_cmsl7ps,
      "\n moyenne pondéré de N_MB_off_ps par duration(s) :", moy_N_MB_duration_off_ps,)

Valeur des différentes moyennes de N_MB_on (méthode online seulement) : 
 
 moyenne non pondéré de N_MB_on             : 1324905626.4074075 
 moyenne pondéré de N_MB_on par cint7l0b    : 1934985968.1328604 
 moyenne pondéré de N_MB_on par cmul7l0b    : 1935292845.6393232 
 moyenne pondéré de N_MB_on par duration(s) : 1933736135.3153353 
 

Valeur des différentes moyennes de N_MB_off_all (méthode offline all + duration) : 
 
 moyenne non pondéré de N_MB_off_all             : 875983449.6055784 
 moyenne pondéré de N_MB_off_all par cint7all    : 1264644389.5275526 
 moyenne pondéré de N_MB_off_all par cmul7all    : 1267719114.384637 
 moyenne pondéré de N_MB_off_all par cmsl7all    : 1268380936.4401455 
 moyenne pondéré de N_MB_off_all par duration(s) : 1262725226.4335728 
 

Valeur des différentes moyennes de N_MB_off_ps (méthode offline ps + duration) : 
 
 moyenne non pondéré de N_MB_off_ps             : 822857743.946547 
 moyenne pondéré de N_MB_off_ps par cint7ps     : 1190108574.419